In [1]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


### Large Datasets and where to find them

#### Challenges of building a large-scale corpus

In [ ]:
from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"GPT  size: {model_size(generation_gpt.model) /1024**2:.1f} M parameters")
print(f"GPT2 size: {model_size(generation_gpt2.model)/1024**2:.1f} M parameters")

In [ ]:
set_seed(1)

In [ ]:
def enum_pipeline_outputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences,
               clean_up_tokenization_spaces=True)

    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

prompt = "\nwhen they came back"
print("GPT completions: \n" + enum_pipeline_outputs(generation_gpt, prompt, 3))
print("")
print("GPT-2 completions: \n" + enum_pipeline_outputs(generation_gpt2, prompt, 3))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT completions: 
1.
when they came back, she 'd want to know what he was going to do. 
 " it 'll teach your brother a lesson, " he said, and then he looked pointedly at my arm and the blood he 'd poured over my skin. "
2.
when they came back, a little boy with a black nose and brown eyes who had started hanging around the neighborhood on special occasions and who lived off of the property. he wore a football jersey with the letters " the black diamond " written on it.
3.
when they came back, and they were still there. " 
 jake nodded. 
 i shrugged and sat back down. " so then, they did come back, " i said, thinking. " but... " 
 " what's her name?

GPT-2 completions: 
1.
when they came back the body was still on the floor and there were a lot of people in there and we tried to hold and push them into the ground and I was just running. But I couldn't help but come out and hit them just
2.
when they came back to the door, they came back.

They come back to the door, and what they said t

### Building a Custom Code Dataset

Creating a Dataset with Google BigQuery

##### the query for the google cloud bigquery

```
SELECT
  f.repo_name, f.path, c.copies, c.size, c.content, l.license
FROM
  bigquery-public-data.github_repos.files AS f
JOIN
  bigquery-public-data.github_repos.contents AS c
ON
  f.id = c.id
JOIN
  bigquery-public-data.github_repos.licenses AS l
ON
  f.repo_name = l.repo_name
WHERE
  NOT c.binary
  AND ((f.path LIKE '%.py')
    AND(c.size BETWEEN 1024
      AND 1048575))
```



##### To get get frin Google cloud storage

```
$ gsutil -m -o "GSUtil:parallel_process_count=1" cp -r gs://<name_of_bucket>
```

#### download directly from huggingface

```
$ git clone https://huggingdace.co/datasets/transformersbook/codeparrot
```

#### Download using datasets

In [ ]:
# Need almost 183.68 GB

# from datasets import load_dataset, DownloadConfig

# download_config = DownloadConfig(delete_extracted=True)
# dataset = load_dataset("./codeparrot", split="train",
#                        download_config=dowinload_config)

In [ ]:
# import psutil
# import os

# print(f"Number of python files code in dataset: {len(dataset)}")
# ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_file)

# # os.stat.st_size is expressed in bytes, so we convert to GB
# print(f"Dataset size (cache file): {ds_size / 2**30:.2f} GB")

# # Process.memory_info is expressed in bytes, so we convert to MB
# print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20} MB")

### Streaming

In [ ]:
# from datasets import load_dataset

# streamed_dataset = load_dataset("./codeparrot", split="train", streaming=True)

In [ ]:
# iterator = iter(streamed_dataset)

# print(dataset[0] == next(iterator))
# print(dataset[1] == next(iterator))

### Pushing dataset to hugging face hub

```
$ huggingface-cli login

$ huggingface-cli repo create --type dataset --organization transformersbook\codeparrot-train ### omit if you are not an organization
$ huggingface-cli repo create --type dataset --organization transformersbook\codeparrot-valid ### omit if you are not an organization

$ git clone https://huggingface.co/datasets/transformersbook/codeparrot-train
$ git clone https://huggingface.co/datasets/transformersbook/codeparrot-valid

$ cd codeparrot-train
$ cp ../codeparrot/*.json.gz .
$ rm ./file-000000000183.json.gz

$ git add .
$ git commit -m "Adding dataset files"
$ git push

$ git ../codeparrot-valid
$ cp ../codeparrot/file-000000000183.json.gz .
$ mv ./file-000000000183.json.gz ./file-000000000183_validation.json.gz
$ git add .
$ git commit -m "Adding dataset files"
$ giit push
```

### Streaming form the huggingface dataset directly

In [ ]:
from datasets import load_dataset

remote_dataset = load_dataset("transformersbook/codeparrot", split="train",
                              streaming=True)

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

### Building a Tokenizer

In [ ]:
from transformers import AutoTokenizer

def tok_list(tokenizer, string):
    input_ids = tokenizer(string, add_special_tokens=False)["input_ids"]
    return [tokenizer.decode(tok) for tok in input_ids]

tokenizer_T5 = AutoTokenizer.from_pretrained("t5-base")
tokenizer_camembert = AutoTokenizer.from_pretrained("camembert-base")

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

In [ ]:
print(f'T5 tokens for "sex": {tok_list(tokenizer_T5, "sex")}')
print(f'CamemBERT tokens for "begin": {tok_list(tokenizer_camembert, "begin")}')

T5 tokens for "sex": ['', 's', 'ex']
CamemBERT tokens for "begin": ['be', 'gin']


### The Tokenizer Model

#### A Tokenizer for Python

In [ ]:
from transformers import AutoTokenizer

python_code = """def say_hello():
    print("Hello, World!")
# Print it
say_hello()
"""

tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


In [ ]:
print(tokenizer.backend_tokenizer.normalizer)

None


In [ ]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


In [ ]:
a, e = u"a", u"€"
byte = ord(a.encode("utf-8"))
print(f'"{a}" is encoded as "{a.encode("utf-8")}" with a single byte: {byte}')
byte = [ord(chr(i)) for i in e.encode("utf-8")]
print(f'{e} is encoded as "{e.encode("utf-8")}" with three bytes: {byte}')

"a" is encoded as "b'a'" with a single byte: 97
€ is encoded as "b'\xe2\x82\xac'" with three bytes: [226, 130, 172]


In [ ]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

bytes_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in bytes_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

print(f'Size of our base vocabulary: {len(base_vocab)}')
print(f'First element: "{base_vocab[0]}", last element "{base_vocab[-1]}"')

Size of our base vocabulary: 256
First element: "!", last element "Ń"


In [ ]:
# id unicode_mapping
# caption Examples of character mappings in BPE

import pandas as pd
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_bytes_map = dict((v, k) for k, v in byte_to_unicode_map.items())

base_vocab = list(unicode_to_bytes_map.keys())

examples = [
    ['Regular characters', '`a` and `?`', f'{ord("a")} and {ord("?")}' , f'`{byte_to_unicode_map[ord("a")]}` and `{byte_to_unicode_map[ord("?")]}`'],
    ['Nonprintable control character (carriage return)', '`U+000D`', f'13', f'`{byte_to_unicode_map[13]}`'],
    ['A space', '` `', f'{ord(" ")}', f'`{byte_to_unicode_map[ord(" ")]}`'],
    ['A nonbreakable space', '`\xa0`', '160', f'`{byte_to_unicode_map[ord(chr(160))]}`'],
    ['A newline character', '`\n`', '10', f'`{byte_to_unicode_map[ord(chr(10))]}`'],
]

pd.DataFrame(examples,
             columns=['Description', 'Character', 'Bytes', 'Mapped bytes'])


,Description,Character,Bytes,Mapped bytes
0,Regular characters,`a` and `?`,97 and 63,`a` and `?`
1,Nonprintable control character (carriage return),`U+000D`,13,`č`
2,A space,` `,32,`Ġ`
3,A nonbreakable space,` `,160,`ł`
4,A newline character,`\n`,10,`Ċ`


In [ ]:
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(python_code))

[('def', (0, 3)), ('Ġsay', (3, 7)), ('_', (7, 8)), ('hello', (8, 13)), ('():', (13, 16)), ('ĊĠĠĠ', (16, 20)), ('Ġprint', (20, 26)), ('("', (26, 28)), ('Hello', (28, 33)), (',', (33, 34)), ('ĠWorld', (34, 40)), ('!")', (40, 43)), ('Ċ', (43, 44)), ('#', (44, 45)), ('ĠPrint', (45, 51)), ('Ġit', (51, 54)), ('Ċ', (54, 55)), ('say', (55, 58)), ('_', (58, 59)), ('hello', (59, 64)), ('()', (64, 66)), ('Ċ', (66, 67))]


In [ ]:
print(f"Size of the vocabulary: {len(tokenizer)}")

Size of the vocabulary: 50257


In [ ]:
print(tokenizer(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!"', ')', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


### Training a Tokenizer

In [ ]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: x[1], reverse=True)
print([f'{tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[:8]]);

['<|endoftext|>', ' gazed', ' informants', ' Collider', ' regress', 'ominated', ' amplification', 'Compar']


In [ ]:
from tqdm.auto import tqdm

length=100000
dataset_name = 'transformersbook/codeparrot-train'
dataset = load_dataset(dataset_name, split="train", streaming=True)
iter_dataset = iter(dataset)

def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['content'] for _ in range(batch_size)]

new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(),
                                                  vocab_size=12500,
                                                  inital_aphabet=base_vocab)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print([f'{tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[257:280]])

['  ', '    ', '   ', '        ', 'se', 'in', '       ', 're', 'on', 'te', '\n       ', '\n        ', 'or', 'st', 'de', '\n   ', 'th', 'le', ' =', 'lf', 'self', 'me', 'al']


In [ ]:
print([f'{new_tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[-12:]])

[' capt', ' embedded', ' regarding', 'Bundle', '355', ' recv', ' dmp', ' vault', ' Mongo', ' possibly', 'implementation', 'Matches']


In [ ]:
print(new_tokenizer(python_code).tokens())

['def', 'Ġs', 'ay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWor', 'ld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 's', 'ay', '_', 'hello', '()', 'Ċ']


In [ ]:
import keyword

print(f"There are in total {len(keyword.kwlist)} Python keywords.")
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer.vocab:
        print(f"No, keyword '{keyw}' is not in the vocabulary")

There are in total 35 Python keywords.
No, keyword 'await' is not in the vocabulary
No, keyword 'finally' is not in the vocabulary
No, keyword 'nonlocal' is not in the vocabulary


In [ ]:
# hide_output
length = 200000
new_tokenizer_larger = tokenizer.train_new_from_iterator(batch_iterator(),
                                                         vocab_size=32768,
                                                         initial_alphabet=base_vocab)



  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
tokens = sorted(new_tokenizer_larger.vocab.items(), key=lambda x: x[1], reverse=False)

print([f'{tokenizer.convert_tokens_to_string([t])}' for t, _ in tokens[-12:]])

[" '<?", 'Functional', ' Images', 'encoders', ' bibrec', ' OPTIONAL', ' rdclass', 'SocketAddressTag', '资金', 'DEPLOYMENT', '经纪公司代码', ")'],"]


In [ ]:
print(new_tokenizer_larger(python_code).tokens())

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


In [ ]:
for keyw in keyword.kwlist:
    if keyw not in new_tokenizer_larger.vocab:
        print(f"No, keyword '{keyw}' is not in vocabulary")

No, keyword 'nonlocal' is not in vocabulary


### Saving a Custom Tokenizer on the Hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
model_ckpt = "codeparrot"
# org = ""
new_tokenizer_larger.push_to_hub(model_ckpt,
                                 # organization=org
                                 )

CommitInfo(commit_url='https://huggingface.co/MohamedAhmedAE/codeparrot/commit/012df1c28f649cfa8bf213b8c01108c8ca4eaf37', commit_message='Upload tokenizer', commit_description='', oid='012df1c28f649cfa8bf213b8c01108c8ca4eaf37', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# reloaded_tokenizer = AutoTokenizer.from_pretrained(org+"/"+model_ckpt)
reloaded_tokenizer = AutoTokenizer.from_pretrained("MohamedAhmedAE/"+model_ckpt)
print(reloaded_tokenizer(python_code).tokens())

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/496k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

['def', 'Ġsay', '_', 'hello', '():', 'ĊĠĠĠ', 'Ġprint', '("', 'Hello', ',', 'ĠWorld', '!")', 'Ċ', '#', 'ĠPrint', 'Ġit', 'Ċ', 'say', '_', 'hello', '()', 'Ċ']


In [ ]:
new_tokenizer.push_to_hub(model_ckpt+"-small-vocabulary",
                          # organization=org
                          )

CommitInfo(commit_url='https://huggingface.co/MohamedAhmedAE/codeparrot-small-vocabulary/commit/755c93af1cdb68e77b7a4be0b374748c1cc6cdb3', commit_message='Upload tokenizer', commit_description='', oid='755c93af1cdb68e77b7a4be0b374748c1cc6cdb3', pr_url=None, pr_revision=None, pr_num=None)

### Training a Model From Scratch

#### Causal language modeling
##### Input: text -> Decoder -> label:text <hr>
#### Masked language modeling
##### Input: text -> Encoder -> label:text <hr>
#### Sequence-to-sequence language modeling
##### Input: text -> Encoder -> Decoder -> label:text <hr>


#### Initializing Model

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_ckpt = "codeparrot"
org = ""
tokenizer = AutoTokenizer.from_pretrained("MohamedAhmedAE/"+model_ckpt)
config = AutoConfig.from_pretrained("gpt2-xl", vocab_size=len(tokenizer))
model = AutoModelForCausalLM.from_config(config)

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/496k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1600, out_features=32768, bias=False)
)

In [ ]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

In [ ]:
print(f"GPT-2 (xl) size: {model_size(model)/1000**2:.1f}M parameters")

GPT-2 (xl) size: 1529.6M parameters


In [ ]:
model.save_pretrained("models/"+model_ckpt, push_to_hub=True,
                      # organization=org
                      )

model-00002-of-00002.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("MohamedAhmedAE/"+model_ckpt)
config_small = AutoConfig.from_pretrained("gpt2", vocab_size=len(tokenizer))
model_small = AutoModelForCausalLM.from_config(config_small)

In [ ]:
print(f"GPT-2 size: {model_size(model_small)/1000**2}M parameters")

GPT-2 size: 111.008256M parameters


In [ ]:
model_small.save_pretrained("models/"+model_ckpt+"-small", push_to_hub=True,
                            # organization=org
                            )

model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

### Implement the Dataloader

In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm

examples, total_characters, total_tokens = 500, 0, 0
dataset = load_dataset("transformersbook/codeparrot-train", split="train",
                       streaming=True)

for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example["content"])
    total_tokens += len(tokenizer(example["content"]).tokens())

characters_per_token = total_characters / total_tokens

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2599 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
print(characters_per_token)

3.6231516195736053


In [ ]:
import torch
from torch.utils.data import IterableDataset

class ConstantLengthDataset(IterableDataset):

    def __init__(self, tokenizer, dataset, seq_length=1024,
                 num_of_sequences=1024, chars_per_token=3.6):

        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.input_characters = seq_length * chars_per_token * num_of_sequences

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.input_characters:
                    m = f"Buffer full: {buffer_len} >= {self.input_characters:.0f}"
                    print(m)
                    break

                try:
                    m=f"Fill buffer: {buffer_len} < {self.input_characters:.0f}"
                    print(m)
                    buffer.append(next(iterator)["content"])
                    buffer_len += len(buffer[-1])

                except StopIteration:
                    iterator = iter(self.dataset)

            all_token_ids = []
            tokenized_inputs = self.tokenizer(buffer, truncation=False)
            for tokenized_input in tokenized_inputs["input_ids"]:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])

            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i: i+self.seq_length]
                if len(input_ids) == self.seq_length:
                    yield torch.tensor(input_ids)


In [ ]:
shuffled_dataset = dataset.shuffle(buffer_size=100)
constant_length_dataset = ConstantLengthDataset(tokenizer, shuffled_dataset,
                                                num_of_sequences=10)

dataset_iterator = iter(constant_length_dataset)

lengths = [len(b) for _,b in zip(range(5), dataset_iterator)]
print(f"Length of sequences: {lengths}")

Fill buffer: 0 < 36864
Fill buffer: 5369 < 36864
Fill buffer: 7521 < 36864
Fill buffer: 9757 < 36864
Fill buffer: 12612 < 36864
Fill buffer: 14293 < 36864
Fill buffer: 21595 < 36864
Fill buffer: 27144 < 36864
Fill buffer: 29313 < 36864
Buffer full: 44574 >= 36864
Length of sequences: [1024, 1024, 1024, 1024, 1024]


### Defining The Training Loop

#### Train with cpu

```
import torch
import torch.nn.functional as F
from datasets import load_dataset

device = "cpu"

model = torch.nn.Transformer().to(device)
optimizer = torch.optim.Adam(model.parameters())
dataset = load_dataset("")
data = torch.utils.data.DataLoader(dataset, shuffle=True)

model.train()
for epoch in range(10):
    for source, targets in data:
        source = source.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        output = model(source)
        loss = F.cross_entropy(output, targets)
        loss.background()
        optimizer.step()
```

### Train with Accelerator

```
import torch
import torch.nn.functional as F
from datasets import load_dataset
from accelerate import Accelerator

accelerator = Accelerator()

model = torch.nn.Transformer()
optimizer = torch.optim.Adam(model.parameters())
dataset = load_dataset("")
data = torch.utils.data.DataLoader(dataset, shuffle=True)
model, optimizer, data = accelerator.prepare(model, optimizer, data)

model.train()
for epoch in range(10):
    for source, targets in data:
        optimizer.zero_grad()
        output = model(source)
        loss = F.cross_entropy(output, targets)
        accelerator.background(loss)
        optimizer.step()

```

In [ ]:
from argparse import Namespace

# Commented parameters correspond to the small model
config = {
    "train_batch_size": 1, #2, # 12
    "valid_batch_size": 1, #2, # 12
    "weight_decay": 0.1,
    "shuffle_buffer": 1000,
    "learning_rate": 2e-4, # 5e-4
    "lr_scheduler_type": "cosine",
    "num_warmup_steps": 3, #750, # 2000
    "gradient_accumulation_steps": 4, #16, # 1
    "max_train_steps": 10, #50000, # 150000
    "max_eval_steps": -1,
    "seq_length": 1024,
    "seed": 1,
    "save_checkpoint_steps": 10 #50000 # 15000
}

In [ ]:
args = Namespace(**config)

In [ ]:
from accelerate import Accelerator
from transformers import set_seed

set_seed(args.seed)

# Accelerator
accelerator = Accelerator()
samples_per_step = accelerator.state.num_processes * args.train_batch_size

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import logging
import datasets
import transformers
import wandb
import os

def setup_logging(project_name):
    os.makedirs("log", exist_ok=True)
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO, handlers=[
            logging.FileHandler(f"log/debug_{accelerator.process_index}.log"),
            logging.StreamHandler()])

    if accelerator.is_main_process: # We onlt want to set up logging one
        wandb.init(project=project_name, config=args)
        run_name = wandb.run.name
        tb_writer = SummaryWriter()
        tb_writer.add_hparams(vars(args), {'0': 0})
        logger.setLevel(logging.INFO)
        datasets.utils.logging.set_verbosity_debug()
        transformers.utils.logging.set_verbosity_info()

    else:
        tb_writer = None
        run_name = ''
        logger.setLevel(logging.ERROR)
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    return logger, tb_writer, run_name

In [ ]:
def log_metrics(step, metrics):
    logger.info(f"Step {step}: {metrics}")
    if accelerator.is_main_process:
        wandb.log(metrics)
        [tb_writer.add_scalar(k, v, step) for k, v in metrics.items()]

In [ ]:
from torch.utils.data.dataloader import DataLoader

def create_dataloaders(dataset_name):
#     train_data = load_dataset(dataset_name+"-train", split="train",
#                               streaming=True)
#     train_data = train_data.shuffle(buffer_size=args.shuffle_buffer,
#                                     seed=args.seed)

#     valid_data = load_dataset(dataset_name+"-valid", split="validation",
#                               streaming=True)
    # -------------------------------------------------------------------------
    # Less Dataset
    train_data = load_dataset("kejian/codesearchnet-python-pep8-v1", split="train")
    train_data = train_data.remove_columns("score")
    train_data = train_data.rename_column("text", "content")
    valid_data = load_dataset("kejian/codesearchnet-python-pep8-v1", split="test")
    valid_data = valid_data.remove_columns("score")
    valid_data = valid_data.rename_column("text", "content")
    # -------------------------------------------------------------------------

    train_dataset = ConstantLengthDataset(tokenizer, train_data,
                                          seq_length=args.seq_length)
    valid_dataset = ConstantLengthDataset(tokenizer, valid_data,
                                          seq_length=args.seq_length)

    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size)
    eval_dataloader = DataLoader(valid_dataset, batch_size=args.valid_batch_size)

    return train_dataloader, eval_dataloader

In [ ]:
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)

        else:
            params_with_wd.append(p)

    return [{'params': params_with_wd, 'weight_decay': args.weight_decay},
            {'params': params_without_wd, 'weight_decay': 0.0}]

In [ ]:
def evaluate():
    model.eval()
    losses = []

    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch, labels=batch)

        loss = outputs.loss.repeat(args.valid_batch_size)
        losses.append(accelerator.gather(loss))
        if args.max_eval_steps > 0 and step >= args.max_eval_steps:
            break

    loss = torch.mean(torch.cat(losses))

    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = torch.tensor(float("inf"))

    return loss.item(), perplexity.item()

In [ ]:
project_name = "MohamedAhmedAE/codeparrot"
dataset_name = 'transformersbook/codeparrot'

In [ ]:
repo_directory = "/content/repo"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from  transformers import AdamW, get_scheduler
from huggingface_hub import Repository

# Logging
logger, tb_writer, run_name = setup_logging(project_name.split("/")[1], )
logger.info(accelerator.state)

# Load model and tokenizer
if accelerator.is_main_process:
    hf_repo = Repository(repo_directory, clone_from=project_name, revision=run_name)

model = AutoModelForCausalLM.from_pretrained(repo_directory)#, gradient_checkpointing=True) # remove it if it missing from congig file
tokenizer = AutoTokenizer.from_pretrained(repo_directory)

# Load dataset and dataloader
train_dataloader, eval_dataloader = create_dataloaders(dataset_name)

# Prepare the optmizer and learning rate scheduler
optimizer = AdamW(get_grouped_params(model), lr=args.learning_rate)
lr_scheduler = get_scheduler(name=args.lr_scheduler_type,
                             optimizer=optimizer,
                             num_warmup_steps=args.num_warmup_steps,
                             num_training_steps=args.max_train_steps)

def get_lr():
    return optimizer.param_groups[0]["lr"]

# Prepare everthing with our "accelerator" (oreder of args is not important)
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model,
                                                                          optimizer,
                                                                          train_dataloader,
                                                                          eval_dataloader)

# Train model
model.train()
completed_steps = 0
for step, batch in enumerate(train_dataloader, start=1):
    loss = model(batch, labels=batch).loss
    log_metrics(step, {"lr": get_lr(),
                       "samples": step*samples_per_step,
                       "steps": completed_steps,
                       "loss/train": loss.item()})

    loss = loss / args.gradient_accumulation_steps
    accelerator.backward(loss)

    if step % args.gradient_accumulation_steps == 0:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1

    if step % args.save_checkpoint_steps == 0:
        logger.info('Evaluating and saving model checkpoint')
        eval_loss, perplexity = evaluate()
        log_metrics(step, {'loss/eval': eval_loss,
                           'perplexity': perplexity})
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)

        if accelerator.is_main_process:
            unwrapped_model.save_pretrained(repo_directory)
            hf_repo.push_to_hub(commit_message=f'step {step}')

        model.train()

    if completed_steps >= args.max_train_steps:
        break

# Evaluate and save the last checkpoint
logger.info("Evaluating and saving model after training")
eval_loss, perplexity = evaluate()
log_metrics(step, {"loss/eval": eval_loss,
                   "perplexity": perplexity})
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
if accelerator.is_main_process:
    unwrapped_model.save_pretrained("./")
    hf_repo.push_to_hub(commit_message=f"final model")

wandb: Currently logged in as: mohamed-ahmed. Use `wandb login --relogin` to force relogin


INFO:__main__:Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse thi

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at /content/repo.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file /content/repo/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

No config specified, defaulting to the single config: codeparrot-train/default
INFO:datasets.builder:No config specified, defaulting to the single config: codeparrot-train/default
Loading Dataset Infos from /usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/json
INFO:datasets.info:Loading Dataset Infos from /usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/json
No config specified, defaulting to the single config: codeparrot-valid/default
INFO:datasets.builder:No config specified, defaulting to the single config: codeparrot-valid/default
Loading Dataset Infos from /usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/json
INFO:datasets.info:Loading Dataset Infos from /usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/json
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.

Fill buffer: 0 < 3774874
Fill buffer: 1950 < 3774874
Fill buffer: 11882 < 3774874
Fill buffer: 13401 < 3774874
Fill buffer: 15850 < 3774874
Fill buffer: 19866 < 3774874
Fill buffer: 21352 < 3774874
Fill buffer: 25311 < 3774874
Fill buffer: 29708 < 3774874
Fill buffer: 31406 < 3774874
Fill buffer: 35468 < 3774874
Fill buffer: 64870 < 3774874
Fill buffer: 75276 < 3774874
Fill buffer: 78118 < 3774874
Fill buffer: 79355 < 3774874
Fill buffer: 80475 < 3774874
Fill buffer: 94065 < 3774874
Fill buffer: 102426 < 3774874
Fill buffer: 105624 < 3774874
Fill buffer: 108968 < 3774874
Fill buffer: 110371 < 3774874
Fill buffer: 112923 < 3774874
Fill buffer: 115016 < 3774874
Fill buffer: 121743 < 3774874
Fill buffer: 138844 < 3774874
Fill buffer: 157430 < 3774874
Fill buffer: 162446 < 3774874
Fill buffer: 165088 < 3774874
Fill buffer: 169183 < 3774874
Fill buffer: 204411 < 3774874
Fill buffer: 205916 < 3774874
Fill buffer: 209353 < 3774874
Fill buffer: 211667 < 3774874
Fill buffer: 214667 < 3774874
Fi

Token indices sequence length is longer than the specified maximum sequence length for this model (2631 > 1024). Running this sequence through the model will result in indexing errors


OutOfMemoryError: CUDA out of memory. Tried to allocate 200.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 175.06 MiB is free. Process 126531 has 14.57 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 17.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
# !pip3 freeze > requirements.txt

### The Training Run With py script
```
$ git clone https://huggingface.co/MohamedAhmedAE/codeparropt
$ cd codeparrot
$ pip install -r requirements.txt
$ wandb login
$ accelerate config
$ accelerate launch codeparrot_training.py
```

#### Accelerate config
<ul>
<li> Compute enviroment ? = multi-GPU
<li> How many machines ? = 1
<li> DeepSpeed ? = No
<li> How many processes ? = 16
<li> Use FP16 ? = 16
</ul>

```
$ git checkout main
$ git merge <RUN_NAME>
$ git push
```

### Results And Analysis

In [2]:
from transformers import pipeline, set_seed

model_ckpt = "transformersbook/codeparrot-small"
generation = pipeline("text-generation", model=model_ckpt, device=0)

config.json:   0%|          | 0.00/865 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [1]:
import re
from transformers import set_seed

def first_block(string):
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

In [2]:
def complete_code(pipe, prompt, max_length=64, nun_completions=4, seed=1):
    set_seed(seed)
    gen_kwargs = {"temperature": 0.4,
                  "top_p": 0.95,
                  "top_k": 0,
                  "num_beams": 1,
                  "do_sample": True}

    code_gens = generation(prompt, num_return_sequences=nun_completions,
                           max_length=max_length, **gen_kwargs)

    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen["generated_text"][len(prompt):])
        code_strings.append(generated_code)

    print(('\n'+'='*80 + '\n').join(code_strings))

In [8]:
prompt = '''def area_of_rectangle(a: float, b: float):
            """Return the area of the rectangle."""'''

complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    return math.sqrt(a * b)

    return a * b / 2.0

    return a * b / 2.0

    return a * b / 2.0


In [9]:
prompt = '''def get_urls_from_html(html):
            """Get all embedded URLs in a HTML string."""'''

complete_code(generation, prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



            return re.findall(r'<a href="(.*?)"', html)

        def get_urls_from_latex(latex):
            """Get all embedded URLs in LaTeX."""
            return re.findall(

            return [url for url in re.findall(r'<a href="(.*?)"', html)]

        def _get_urls_from_html(html):
            """Get all embedded URLs in a HTML

            return [
                url for url in re.findall(r'<a href="(.*?)"', html)
                if url.endswith('.html')
            ]

    def get_urls(self, page):

            if not isinstance(html, basestring):
                html = str(html)
            return [
                url_parse(url)
                for url in re.findall(
                    r'<a href="(.*?)"',


In [11]:
import requests

def get_urls_from_html(html):
    return [url for url in re.findall(r'<a href="(.*?)"', html) if url]

print('|'. join(get_urls_from_html(requests.get("https://hf.co/").text)))

/models|/spaces/InstantX/InstantID|/spaces/HuggingFaceM4/screenshot2html|/spaces/FaceOnLive/ID-Document-Recognition-SDK|/spaces/lmsys/chatbot-arena-leaderboard|/spaces/Recognito/FaceAnalysis|/spaces|/datasets|/docs/transformers|/docs/diffusers|/docs/safetensors|/docs/huggingface_hub|/docs/tokenizers|/docs/peft|/docs/transformers.js|/docs/timm


In [ ]:
from transformers import pipeline, set_seed

model_ckpt = "transformersbook/codeparrot"

generation = pipeline("text-generation", model=model_ckpt, device=0)

prompt = '''# a function in native python:
def mean(a):
    return sum(a)/len(a)

# the same function using numpy:
import numpy as np
def mean(a):'''

complete_code(generation, prompt, max_length=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
prompt = '''X = np.random.randn(100, 100)
y = np.random.randint(0, 1, 100)

# fit random forest classifier with 20 estimators'''
complete_code(generation, prompt, max_length=96)